# Baseline GAN

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd  drive/MyDrive/tesi2

/content/drive/MyDrive/tesi2


In [ ]:
%pwd

'/content/drive/MyDrive/tesi2'

In [ ]:
%ls .

content/
drive/
dropped/
dropped_impressionist/
GAN.ipynb
gan_landscapes.h5
image_resized128/
images/
images_R/
impressionist_landscape_1024/
impressionist_landscapes/
impressionistlandscapespaintings.zip
impressionist_landscapes_resized_1024/
kaggle.json
Models/
output/
output2/
output_landscapes/
output_landscapes50/
output_landscapes50_rev/
resized128/
resized256/
Results/
the-metropolitan-museum-of-art-ukiyoe-dataset.zip
training_data128_new.npy
training_data128.npy
training_data256_new.npy
training_data_256.npy
training_data256.npy
training_data_landscapes_128.npy
training_data.npy
trn_data.pkl


In [ ]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8 MB 24 kB/s 
     |████████████████████████████████| 3.8 MB 34.8 MB/s 
     |████████████████████████████████| 448 kB 37.1 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=f46a72b51355026bf174ab5bf7cd295173961e263d02b6743a98567a041462a8
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
!pip install keras==2.3.1

     |████████████████████████████████| 377 kB 7.6 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [ ]:
#! kaggle datasets download robgonsalves/impressionistlandscapespaintings


 99% 1.00G/1.00G [00:08<00:00, 139MB/s]
100% 1.00G/1.00G [00:08<00:00, 131MB/s]


In [ ]:
#!unzip /content/drive/MyDrive/tesi2/impressionistlandscapespaintings.zip

Archive:  /content/drive/MyDrive/tesi2/impressionistlandscapespaintings.zip
replace content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_a-copse-evening-1918.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_early-spring-quebec-1923.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import keras
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import PIL
from PIL import Image
import os
import os.path
import pickle


Using TensorFlow backend.


In [ ]:
# Defining an image size and image channel
# We are going to resize all our images to 128X128 size and since the images are colored images
# We are setting our image channels to 3 (RGB)

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = '/content/drive/MyDrive/tesi2/impressionist'

# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR

training_data = []

# Iterating over the images inside the directory and resizing them using
# Pillow's resize method.
print('resizing...')
"""
for filename in os.listdir(images_path):
  path = os.path.join(images_path, filename)
  im = Image.open(path)
  image = im.convert('RGB')
  fake_data.append(np.asarray(image))

for filename in os.listdir(images_path):
  path = os.path.join(images_path, filename)
  im = Image.open(path)
  image = im.convert('RGB')
  true_data.append(np.asarray(image))  


"""

resizing...


"\nfor filename in os.listdir(images_path):\n  path = os.path.join(images_path, filename)\n  im = Image.open(path)\n  image = im.convert('RGB')\n  fake_data.append(np.asarray(image))\n\nfor filename in os.listdir(images_path):\n  path = os.path.join(images_path, filename)\n  im = Image.open(path)\n  image = im.convert('RGB')\n  true_data.append(np.asarray(image))  \n\n\n"

In [ ]:
!git clone https://github.com/t0nberryking/DCGAN256.git

Cloning into 'DCGAN256'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (30/30), done.


In [ ]:
"""
# image_resizer.py

# Defining an image size and image channel
# We are going to resize all our images to 128X128 size and since our images are colored images
# We are setting our image channels to 3 (RGB)

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = '/content/drive/MyDrive/tesi2/resized256'

# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR

training_data = []

# Iterating over the images inside the directory and resizing them using
# Pillow's resize method.
print('resizing...')

try:
    for artist in os.listdir(images_path):
      artist_path = os.path.join(images_path,artist)
      for file in os.listdir(artist_path):
        path = os.path.join(artist_path,file)      
        if os.stat(path).st_size == 3727:
            pass
        else:
            im = Image.open(path)
            image = im.convert('RGB')
            image = image.resize(
                (IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

            training_data.append(np.asarray(image))
except:
    pass

"""

resizing...


In [ ]:
len(training_data)

3313

In [ ]:
training_data = np.reshape(training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
training_data = training_data/255
np.save(os.path.join('/content/drive/MyDrive/tesi2','training_data_landscapes_128'), training_data)

In [ ]:
# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 1
SAVE_FREQ = 500

# Size vector to generate images from
NOISE_SIZE = 123

# Configuration
EPOCHS = 50001 # number of iterations
BATCH_SIZE = 50

GENERATE_RES = 3
IMAGE_SIZE = 128  # rows/cols

IMAGE_CHANNELS = 3
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

#training_data = np.load('/content/drive/MyDrive/tesi2/training_data128.npy')

In [ ]:
def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [ ]:
build_discriminator(image_shape).summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 1720385   
Total params: 1,720,385
Trainable params: 1,718,465
Non-trainable params: 1,920
_________________________________________________________________


In [ ]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu', input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    for i in range(GENERATE_RES):
         model.add(UpSampling2D())
         model.add(Conv2D(256, kernel_size=3, padding='same'))
         model.add(BatchNormalization(momentum=0.8))
         model.add(Activation('relu'))
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    
    return Model(input, generated_image)

In [ ]:
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4096)              507904    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_13 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 16, 16, 256)      

In [ ]:
discriminator = build_discriminator(image_shape)

In [ ]:
discriminator.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
sequential_6 (Sequential)    (None, 1)                 1720385   
Total params: 1,720,385
Trainable params: 1,718,465
Non-trainable params: 1,920
_________________________________________________________________


In [ ]:
def save_images(cnt, noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)

    generated_images = generator.predict(noise)

    #generated_images = 0.5 * generated_images + 0.5

    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1

    output_path = '/content/drive/MyDrive/tesi2/output_landscapes50'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)


In [ ]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

optimizer = Adam(1.5e-4, 0.5)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss="binary_crossentropy",
                      optimizer=optimizer, metrics=["accuracy"])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)


#def wasserstein_loss(y_true, y_pred):
#    """Calculates the Wasserstein loss - critic maximises the distance between its output for real and generated samples.
#    To achieve this generated samples have the label -1 and real samples the label 1. Multiplying the outputs by the labels results to the wasserstein loss via the Kantorovich-Rubinstein duality"""
#    return K.mean(y_true * y_pred)

combined = Model(random_input, validity)
combined.compile(loss="binary_crossentropy",
                 optimizer=optimizer, metrics=["accuracy"])



y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))

fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1

for epoch in range(EPOCHS):
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    x_real = training_data[idx]

    noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)

    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)

    discriminator_metric_generated = discriminator.train_on_batch(
        x_fake, y_fake)

    discriminator_metric = 0.5 * \
        np.add(discriminator_metric_real, discriminator_metric_generated)

    generator_metric = combined.train_on_batch(noise, y_real)
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise)
        cnt += 1

        print(f"{epoch} epoch, Discriminator accuracy: {100* discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}")
        generator.save(os.path.join('/content/drive/MyDrive/tesi2/output_landscapes50', "modern_art_generator.h5"))


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4096)              507904    
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_14 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_7 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 16, 16, 256)      

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 epoch, Discriminator accuracy: 20.000000298023224, Generator accuracy: 100.0


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


500 epoch, Discriminator accuracy: 56.00000023841858, Generator accuracy: 5.999999865889549
1000 epoch, Discriminator accuracy: 62.00000047683716, Generator accuracy: 18.000000715255737
1500 epoch, Discriminator accuracy: 50.999999046325684, Generator accuracy: 5.999999865889549
2000 epoch, Discriminator accuracy: 61.000001430511475, Generator accuracy: 7.999999821186066
2500 epoch, Discriminator accuracy: 56.99999928474426, Generator accuracy: 5.999999865889549
3000 epoch, Discriminator accuracy: 49.000000953674316, Generator accuracy: 3.999999910593033
3500 epoch, Discriminator accuracy: 63.999998569488525, Generator accuracy: 7.999999821186066
4000 epoch, Discriminator accuracy: 63.999998569488525, Generator accuracy: 18.000000715255737
4500 epoch, Discriminator accuracy: 71.00000381469727, Generator accuracy: 15.999999642372131
5000 epoch, Discriminator accuracy: 67.99999475479126, Generator accuracy: 10.000000149011612
5500 epoch, Discriminator accuracy: 52.99999713897705, Generat

In [ ]:
import os 
import os.path
import numpy as np
import PIL
from PIL import Image

In [ ]:
cnt = 0
for file in os.listdir('/content/drive/MyDrive/tesi2/output_landscapes50'):
  path = os.path.join("/content/drive/MyDrive/tesi2/output_landscapes50",file)
  im = Image.open(path)
  image = im.convert('RGB')
  im_array = np.asarray(image)
  im_array = ((im_array/255.0)-0.5)/0.5
  im = Image.fromarray((im_array * 255).astype(np.uint8))
  im.save(os.path.join("/content/drive/MyDrive/tesi2/output_landscapes50_rev",f"trained-{cnt}.png"))
  cnt += 1

In [ ]:
os.listdir('/content/drive/MyDrive/tesi2/output_landscapes50')[0]

'trained-1.png'

In [ ]:
generator.save(os.path.join('/content/drive/MyDrive/tesi2/output', "art_generator.h5"))